# Iteration 3: Sample length of stay on the ASU

## Plain English Summary

In the model length of stay is determined not just by patient type, but also patient destination post ASU.  For example Early Supported Discharge greatly reduces length of stay.  Now that we have sampled the destination we can sample the length of stay from Lognormal distributions.



## Notebook imports

In [1]:
from IPython.display import Markdown, display

## LLM Prompt

In [2]:
display(Markdown("inputs/03_iteration.md"))

### Main command

Modify the model to add in an acute stroke unit (ASU) treatment process.

Show all of the model code: this includes all patient generator functions in the ASU class.

### General model logic

Add the following logic

1. sample a length of stay (measured in days) for treatment of patients
2. yield a simpy timeout equal to the length of stay for treatment

The model should print out useful information after each event.

### Simulation inputs

the length of stay (measured in days) for treatment of patients in the ASU follow source specific distributions

if patient type = Stroke and post_asu_destination = Rehab then Lognormal: mean 7.4 and standard deviation 8.6
if patient type = Stroke and post_asu_destination = ESD then Lognormal: mean 4.6 and standard deviation 4.8
if patient type = Stroke and post_asu_destination = Other then Lognormal: mean 7.0 and standard deviation 8.7

if patient type = TIA then Lognormal: mean 1.8 and standard deviation 5.0
if patient type = Complex Neurological then Lognormal: mean 4.0 and standard deviation 5.0
if patient type = Other then Lognormal: mean 3.8 and standard deviation 5.2

The mean and standard deviation of the Lognormal distributions must be converted to the mean and standard deviation of the underlying normal distribution.




## Date and time of input to LLM

> 1545 10th February 2024

## Observations on the LLM Output

* 

## Sources reported by the LLM

* 

## LLM Output

In [3]:
import simpy
import numpy as np

# Define the discrete-event simulation model
class AcuteStrokeUnit:
    def __init__(self, env, run_length=1825):  # 5 years in days
        self.env = env
        self.stroke_arrival = simpy.Container(env, init=0)
        self.tia_arrival = simpy.Container(env, init=0)
        self.neuro_arrival = simpy.Container(env, init=0)
        self.other_arrival = simpy.Container(env, init=0)
        self.stroke_interarrival_mean = 1.2
        self.tia_interarrival_mean = 9.3
        self.neuro_interarrival_mean = 3.6
        self.other_interarrival_mean = 3.2
        self.run_length = run_length
        self.patient_count = 0

    def stroke_patient_generator(self):
        while True:
            interarrival_time = np.random.exponential(self.stroke_interarrival_mean)
            yield self.env.timeout(interarrival_time)
            self.stroke_arrival.put(1)
            self.patient_count += 1
            post_asu_destination = np.random.choice(["Rehab", "ESD", "Other"], p=[0.24, 0.13, 0.63])
            if post_asu_destination == "Rehab":
                los_mean, los_std = 7.4, 8.6
            elif post_asu_destination == "ESD":
                los_mean, los_std = 4.6, 4.8
            else:
                los_mean, los_std = 7.0, 8.7
            length_of_stay = np.random.lognormal(np.log(los_mean**2 / np.sqrt(los_mean**2 + los_std**2)), np.sqrt(np.log(1 + (los_std**2 / los_mean**2))))
            print(f"Stroke patient {self.patient_count} arrived at {self.env.now} days and will go to {post_asu_destination} for {length_of_stay} days of treatment")
            yield self.env.timeout(length_of_stay)

    def tia_patient_generator(self):
        while True:
            interarrival_time = np.random.exponential(self.tia_interarrival_mean)
            yield self.env.timeout(interarrival_time)
            self.tia_arrival.put(1)
            self.patient_count += 1
            length_of_stay = np.random.lognormal(np.log(1.8**2 / np.sqrt(1.8**2 + 5.0**2)), np.sqrt(np.log(1 + (5.0**2 / 1.8**2))))
            print(f"TIA patient {self.patient_count} arrived at {self.env.now} days for {length_of_stay} days of treatment")
            yield self.env.timeout(length_of_stay)

    def neuro_patient_generator(self):
        while True:
            interarrival_time = np.random.exponential(self.neuro_interarrival_mean)
            yield self.env.timeout(interarrival_time)
            self.neuro_arrival.put(1)
            self.patient_count += 1
            length_of_stay = np.random.lognormal(np.log(4.0**2 / np.sqrt(4.0**2 + 5.0**2)), np.sqrt(np.log(1 + (5.0**2 / 4.0**2))))
            print(f"Complex Neuro patient {self.patient_count} arrived at {self.env.now} days for {length_of_stay} days of treatment")
            yield self.env.timeout(length_of_stay)

    def other_patient_generator(self):
        while True:
            interarrival_time = np.random.exponential(self.other_interarrival_mean)
            yield self.env.timeout(interarrival_time)
            self.other_arrival.put(1)
            self.patient_count += 1
            length_of_stay = np.random.lognormal(np.log(3.8**2 / np.sqrt(3.8**2 + 5.2**2)), np.sqrt(np.log(1 + (5.2**2 / 3.8**2))))
            print(f"Other patient {self.patient_count} arrived at {self.env.now} days for {length_of_stay} days of treatment")
            yield self.env.timeout(length_of_stay)

# Create the simulation environment
env = simpy.Environment()

# Initialize the Acute Stroke Unit model
acu = AcuteStrokeUnit(env)

# Start the patient generators
env.process(acu.stroke_patient_generator())
env.process(acu.tia_patient_generator())
env.process(acu.neuro_patient_generator())
env.process(acu.other_patient_generator())

# Run the simulation
env.run(until=acu.run_length)


Complex Neuro patient 1 arrived at 1.6337847978181639 days for 6.300763221502295 days of treatment
Other patient 2 arrived at 4.305436092713494 days for 0.33324779276535643 days of treatment
Stroke patient 3 arrived at 4.912411514536697 days and will go to ESD for 2.164069919639456 days of treatment
TIA patient 4 arrived at 7.59693961961195 days for 0.4555470178541546 days of treatment
Stroke patient 5 arrived at 7.710933461486852 days and will go to Other for 7.773560717885945 days of treatment
Complex Neuro patient 6 arrived at 8.518310304672363 days for 2.653823267195605 days of treatment
TIA patient 7 arrived at 11.13183589008499 days for 1.407484851097238 days of treatment
Other patient 8 arrived at 11.756651512006798 days for 0.905341244724281 days of treatment
Other patient 9 arrived at 13.976418362713273 days for 1.3634028291918072 days of treatment
Complex Neuro patient 10 arrived at 16.190533848490823 days for 4.778051854806566 days of treatment
Stroke patient 11 arrived at 1

## Testing

* **Extreme value test 1**: TIA, Compelex Neuro, Other, have their inter-arrival time is set to $M$ a very large number
    * Expected result: The only type of patient to arrive to the model is "Stroke"
    * Actual result (PASS): The only type of patient to arrive to the model is "Accident and Emergency."
* **Extreme value test 2**: All patient types have have their inter-arrival time is set to $M$ a very large number
    * Expected result: No patients arrive to the model.
    * Actual result (PASS): No patients arrive to the model.
* **Different run length**: The run length of the model is set to 10 days.
    * Expected result: The model should runs no longer than 240 simulated time periods
    * Actual result (PASS): The model runs no longer than 240 simulated time periods

### Extreme value test 1

The model uses hard coded parameters.  The most simple way to modify the code for this test was to change hard coded parameters for $M$

**Result: PASS**

In [ ]:
M = 10_000_000

In [ ]:
def test_1(large_number):
    # Create the simulation environment
    env = simpy.Environment()
    
    # Initialize the Acute Stroke Unit model
    asu = AcuteStrokeUnit(env)

    #set ASU arrival rates
    asu.stroke_interarrival_mean = 1.2
    asu.tia_interarrival_mean = large_number
    asu.neuro_interarrival_mean = large_number
    asu.other_interarrival_mean = large_number
    
    # Start the patient generators
    env.process(asu.stroke_patient_generator())
    env.process(asu.tia_patient_generator())
    env.process(asu.neuro_patient_generator())
    env.process(asu.other_patient_generator())
    
    # Run the simulation
    env.run(until=asu.run_length)

    print(asu.patient_count)


In [ ]:
test_1(M)

### Extreme value test 2

* The code is again modified so that all patient type also have a mean inter-arrival rate of $M$

**Result: PASS**

In [ ]:
def test_2(large_number):
    # Create the simulation environment
    env = simpy.Environment()
    
    # Initialize the Acute Stroke Unit model
    asu = AcuteStrokeUnit(env)

    #set ASU arrival rates
    asu.stroke_interarrival_mean = large_number
    asu.tia_interarrival_mean = large_number
    asu.neuro_interarrival_mean = large_number
    asu.other_interarrival_mean = large_number
    
    # Start the patient generators
    env.process(asu.stroke_patient_generator())
    env.process(asu.tia_patient_generator())
    env.process(asu.neuro_patient_generator())
    env.process(asu.other_patient_generator())
    
    # Run the simulation
    env.run(until=asu.run_length)

    print(asu.patient_count)


In [ ]:
test_2(M)

### Different run length test

The model has again been modified to use the original code and parameters generated by the LLM.

In [ ]:
TEN_DAYS = 10

In [ ]:
def test_3(run_length):
    # Create the simulation environment
    env = simpy.Environment()
    
    # Initialize the Acute Stroke Unit model
    asu = AcuteStrokeUnit(env, run_length=run_length)
    
    # Start the patient generators
    env.process(asu.stroke_patient_generator())
    env.process(asu.tia_patient_generator())
    env.process(asu.neuro_patient_generator())
    env.process(asu.other_patient_generator())
    
    # Run the simulation
    env.run(until=asu.run_length)

    print(env.now)
    


In [ ]:
test_3(TEN_DAYS)